<a href="https://colab.research.google.com/gist/bhavika1506/d9f26f0d6ba2c6b66d0741e72e21f3d3/stocks_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing all imp libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.backend as K
from keras.optimizers import Adam
from keras.models import load_model
from keras.layers import LSTM
np.random.seed(7)

In [ ]:
# data is of netflix from date-(1-aug-2003)_to_(28-aug-2020) from yahoo finance
df = pd.read_csv("/content/NFLX.csv",header=0)
df = df.sort_index(ascending=True, axis=0)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2003-08-01,1.851429,1.857143,1.782143,1.782143,1.782143,6339200
1,2003-08-04,1.782143,1.797143,1.735714,1.782143,1.782143,3676400
2,2003-08-05,1.792857,1.817857,1.732857,1.736429,1.736429,3084200
3,2003-08-06,1.742857,1.750000,1.633571,1.668571,1.668571,8113000
4,2003-08-07,1.650714,1.667857,1.614286,1.649286,1.649286,7893200


In [ ]:
df.shape

(4299, 7)

In [ ]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,4299.000000,4299.000000,4299.000000,4299.000000,4299.000000,4.299000e+03
mean,85.137456,86.506299,83.731468,85.187151,85.187151,1.816332e+07
std,123.252283,125.243608,121.193015,123.337870,123.337870,2.003304e+07
min,1.300000,1.317143,1.272857,1.290000,1.290000,1.493800e+06
25%,4.143572,4.220000,4.067858,4.142857,4.142857,7.197400e+06
50%,22.384285,22.858572,21.722857,22.245714,22.245714,1.196440e+07
75%,105.120003,107.360000,102.685001,105.280002,105.280002,2.165660e+07
max,567.979980,575.369995,521.250000,548.729980,548.729980,3.234140e+08


In [ ]:
df.isnull().any()

Date         False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [ ]:
fig = px.line(df, x='Date', y='Close')
fig.show()

In [ ]:
# Taking diff indicators for prediction
# ohlc_avg is the average of open, high, low, close values
# hlc_avgs is the average of high, low, close value
# we will take only ohlc_avg data only in whole nb
ohlc_data = df.iloc[:, 1:5]
ohlc_avg = ohlc_data.mean(axis=1)
hlc_avg = df[['High', 'Low', 'Close']].mean(axis=1)
close = df.Close

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x = df.index, y = ohlc_avg,
                  name='OHLC avg'))
fig1.add_trace(go.Scatter(x = df.index, y = hlc_avg,
                  name='HLC avg'))
fig1.add_trace(go.Scatter(x = df.index, y = close,
                  name='close column data'))
fig1.show()

In [ ]:
pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 14.2 MB/s eta 0:00:00


In [ ]:
if not os.path.exists("images"):
    os.mkdir("images")


In [ ]:
# we will create a new df which has only 2 column which is useful to predict data
new_data = pd.DataFrame(index=range(0,len(df)), columns=['Date', 'ohlc_avg'])
for i in range(0, len(df)):
  new_data['Date'][i] = df['Date'][i]
  new_data['ohlc_avg'][i] = ohlc_avg[i]

In [ ]:
new_data.head()

,Date,ohlc_avg
0,2003-08-01,1.818214
1,2003-08-04,1.774286
2,2003-08-05,1.77
3,2003-08-06,1.69875
4,2003-08-07,1.645536


In [ ]:
# setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

In [ ]:
print(len(new_data))

4299


In [ ]:
ds = new_data.values

In [ ]:
# we will take 80% data in train and remaining in test
train = int(len(new_data)*0.8)
test = len(new_data) - train
train, test = new_data.iloc[0:train,:], new_data.iloc[train:len(new_data),:]

In [ ]:
train.shape

(3439, 1)

In [ ]:
# we have normalize the data cuz data is like 149...., 488..something like that
# so we have to normalize betwwen 0 and 1
scalar = MinMaxScaler(feature_range=(0, 1))
scaled_data = scalar.fit_transform(ds)

In [ ]:
# splitting the data to x_train, y_train
# we will first train upto 60 and then predict on 61 and then
# we will train from 61 to 120 then predict on 121 likewise we will go
x_train, y_train = [], []
for i in range(60, len(train)):
  x_train.append(scaled_data[i-60:i,0])
  y_train.append(scaled_data[i,0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train.shape

(3379, 60)

In [ ]:
# create and fit the lstm network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.25))
model.add(LSTM(units=50))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
106/106 [==============================] - 12s 74ms/step - loss: 2.8740e-04
Epoch 2/50
106/106 [==============================] - 7s 61ms/step - loss: 4.3483e-05
Epoch 3/50
106/106 [==============================] - 8s 73ms/step - loss: 4.5883e-05
Epoch 4/50
106/106 [==============================] - 6s 61ms/step - loss: 3.4161e-05
Epoch 5/50
106/106 [==============================] - 8s 72ms/step - loss: 3.5198e-05
Epoch 6/50
106/106 [==============================] - 6s 61ms/step - loss: 3.5540e-05
Epoch 7/50
106/106 [==============================] - 8s 73ms/step - loss: 3.2197e-05
Epoch 8/50
106/106 [==============================] - 7s 68ms/step - loss: 2.9132e-05
Epoch 9/50
106/106 [==============================] - 13s 121ms/step - loss: 3.0264e-05
Epoch 10/50
106/106 [==============================] - 8s 77ms/step - loss: 3.4700e-05
Epoch 11/50
106/106 [==============================] - 9s 87ms/step - loss: 3.1212e-05
Epoch 12/50
106/106 [============================

In [ ]:
# predicting 920 values, using past 60 from the train data
inputs = new_data[len(new_data)-len(test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = scalar.transform(inputs)

In [ ]:
inputs.shape

(920, 1)

In [ ]:
x_test = []
for i in range(60,inputs.shape[0]):
    x_test.append(inputs[i-60:i,0])
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predicted_price = model.predict(x_test)
# inverse transform for getting back all normal values from scaled values
predicted_price = scalar.inverse_transform(predicted_price)

27/27 [==============================] - 1s 18ms/step


In [ ]:
rms=np.sqrt(np.mean(np.power((test-predicted_price),2)))
rms

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning:

In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'



ohlc_avg    9.609548
dtype: float64

In [ ]:
# create a new column of predicted values
test['Prediction'] = predicted_price
test.head()

<ipython-input-45-db9a93f10bcc>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ohlc_avg,Prediction
Date,,
2017-03-30,147.422497,149.003754
2017-03-31,147.842499,150.554459
2017-04-03,146.842499,151.671097
2017-04-04,146.079998,151.810501
2017-04-05,144.829998,151.313934


In [ ]:
# Graph for comparing the results of model predicted and original value
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x = train.index, y = train.ohlc_avg,
                  name='train'))
fig2.add_trace(go.Scatter(x = test.index, y = test.ohlc_avg,
                  name='test_ohlc_avg'))
fig2.add_trace(go.Scatter(x = test.index, y = test.Prediction,
                  name='test'))
fig2.show()

In [ ]:
pip install -U kaleido

In [ ]:
fig3 = px.line(df, x='Date', y='Close')
fig3.show()